# Eat Safe, Love

## Notebook Set Up

In [14]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [15]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [16]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [17]:
# review the collections in our database
pprint(db.establishments.find_one())

{'AddressLine1': '146A Plumstead Rd',
 'AddressLine2': '',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67a96a230cd9e8814c5eec58'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [18]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [19]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with Hygiene score 20: {count}")

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

Number of establishments with Hygiene score 20: 0
None


In [20]:
# Convert the result to a Pandas DataFrame
hygiene_score_df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(hygiene_score_df)}")

# Display the first 10 rows of the DataFrame
hygiene_score_df.head(10)

Number of rows: 0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [21]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London"}, "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {count}")

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

Number of establishments in London with RatingValue >= 4: 0
None


In [22]:
# Convert the result to a Pandas DataFrame
london_ratings_df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(london_ratings_df)}")

# Display the first 10 rows of the DataFrame
london_ratings_df.head(10)

Number of rows: 0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [23]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
}
sort = [("scores.Hygiene", 1)]
limit = 5

# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))
pprint(results)

[]


In [24]:
# Convert result to Pandas DataFrame
top_5_df = pd.DataFrame(results)
top_5_df

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [25]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
]

# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))

# Print the first 10 results
pprint(results[:10])

[]


In [26]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
num_rows_df = pd.DataFrame({'Number of Rows': [hygiene_df.shape[0]]})
display(num_rows_df)

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

,Number of Rows
0,0


""


### Troubleshooting 0 values

In [28]:
query = {"scores.Hygiene": 20}
query

{'scores.Hygiene': 20}

In [29]:
hygiene_values = establishments.distinct("scores.Hygiene")
print(hygiene_values)

['']


In [30]:
query = {"LocalAuthorityName": {"$regex": "London"}, "RatingValue": {"$gte": 4}}
query

{'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte': 4}}

In [31]:
rating_values = establishments.distinct("RatingValue")
print(rating_values)

[]


In [33]:
query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
}
query

{'RatingValue': 5,
 'geocode.latitude': {'$gte': 51.480142, '$lte': 51.500142},
 'geocode.longitude': {'$gte': 0.07384, '$lte': 0.09383999999999999}}

In [34]:
sample_geocode = establishments.find_one({}, {"geocode": 1, "_id": 0})
pprint(sample_geocode)

{'geocode': {'latitude': 51.490142, 'longitude': 0.08384}}


In [35]:
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
]
pipeline

[{'$match': {'scores.Hygiene': 0}},
 {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}},
 {'$sort': {'count': -1}}]

In [36]:
zero_hygiene = establishments.find({"scores.Hygiene": 0})
pprint(list(zero_hygiene))

[]


In [37]:
count = establishments.count_documents({})
print(f"Total documents in establishments: {count}")

Total documents in establishments: 4


In [38]:
query = {"scores.Hygiene": {"$ne": ""}}
hygiene_values = establishments.distinct("scores.Hygiene", query)
print(hygiene_values)

[]


In [40]:
query = {"scores.Hygiene": {"$ne": ""}}  # Ensure that it’s not an empty string
hygiene_score_df = pd.DataFrame(list(establishments.find(query)))
hygiene_score_df

""


In [43]:
pipeline = [
    {"$match": {"scores.Hygiene": {"$ne": ""}}},  # Exclude empty hygiene scores
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
]
results = list(establishments.aggregate(pipeline))
pprint(results)

[]


In [44]:
latitudes = establishments.distinct("geocode.latitude")
longitudes = establishments.distinct("geocode.longitude")
print(latitudes, longitudes)

[51.490142] [0.08384]


In [45]:
count = establishments.count_documents({"scores.Hygiene": {"$ne": ""}})
print(f"Count of establishments with non-empty hygiene score: {count}")

Count of establishments with non-empty hygiene score: 0


In [46]:
hygiene_values = establishments.distinct("scores.Hygiene")
print("Distinct Hygiene Scores:", hygiene_values)

rating_values = establishments.distinct("RatingValue")
print("Distinct Rating Values:", rating_values)

Distinct Hygiene Scores: ['']
Distinct Rating Values: []


In [48]:
query = {"scores.Hygiene": {"$gte": 0}}  # Include values greater than or equal to 0 (or handle missing values).
query

{'scores.Hygiene': {'$gte': 0}}

In [49]:
query = {"scores.Hygiene": {"$ne": ""}}  # Exclude empty strings
query

{'scores.Hygiene': {'$ne': ''}}

In [50]:
query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - 0.05, "$lte": latitude + 0.05},
    "geocode.longitude": {"$gte": longitude - 0.05, "$lte": longitude + 0.05},
}
query

{'RatingValue': 5,
 'geocode.latitude': {'$gte': 51.440142, '$lte': 51.540141999999996},
 'geocode.longitude': {'$gte': 0.033839999999999995,
  '$lte': 0.13384000000000001}}

In [51]:
hygiene_values = establishments.distinct("scores.Hygiene")
print(hygiene_values)

['']


In [52]:
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))

{'AddressLine1': '146A Plumstead Rd',
 'AddressLine2': '',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67a96a230cd9e8814c5eec58'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [53]:
establishments.count_documents({"scores.Hygiene": {"$in": ["", None]}})

5